# Add Public Raster Dataset

In [1]:
import geoengine as ge
from datetime import datetime

## Initialize Geo Engine as Admin

In [2]:
ge.initialize("http://localhost:3030", admin_token="8aca8875-425a-4ef1-8ee6-cdfc62dd7525")

## List volumes

In [3]:
volumes = ge.volumes()
volumes

[Volume(name='test_data', path='test_data/')]

## Define metadata

In [4]:
file_path = "raster/landcover.tif" # specify file path relative to volume path

rasterband_channel = 1

origin = [-180.0, 90.0]
x_pixel_size = 0.1
y_pixel_size = -0.1
geo_transform = ge.GdalDatasetGeoTransform({
  "originCoordinate": origin, 
  "xPixelSize": x_pixel_size, 
  "yPixelSize": y_pixel_size
})

width = 3600
height = 1800

file_not_found_handling = ge.FileNotFoundHandling.NODATA

gdal_params = ge.GdalDatasetParameters({
  "filePath": file_path, 
  "rasterbandChannel": rasterband_channel, 
  "geoTransform": geo_transform,
  "width": width, 
  "height": height , 
  "fileNotFoundHandling": file_not_found_handling, 
  "noDataValue": None, 
  "propertiesMapping": None, 
  "gdalOpenOptions": None, 
  "gdalConfigOptions": None, 
  "allowAlphabandAsMask": True
})

result_descriptor_dict = {
      "dataType": "U8",
      "spatialReference": "EPSG:4326",
      "measurement": {
        "type": "classification",
        "measurement": "Land Cover",
        "classes": {
          "0": "Water Bodies",
          "1": "Evergreen Needleleaf Forests",
          "2": "Evergreen Broadleaf Forests",
          "3": "Deciduous Needleleaf Forests",
          "4": "Deciduous Broadleleaf Forests",
          "5": "Mixed Forests",
          "6": "Closed Shrublands",
          "7": "Open Shrublands",
          "8": "Woody Savannas",
          "9": "Savannas",
          "10": "Grasslands",
          "11": "Permanent Wtlands",
          "12": "Croplands",
          "13": "Urban and Built-Up",
          "14": "Cropland-Natural Vegetation Mosaics",
          "15": "Snow and Ice",
          "16": "Barren or Sparsely Vegetated"
        }
      },
      "noDataValue": 255.0,
      "time": {
        "start": "-262144-01-01T00:00:00+00:00",
        "end": "+262143-12-31T23:59:59.999+00:00"
      },
      "bbox": {
        "upperLeftCoordinate": [-180.0, 90.0],
        "lowerRightCoordinate": [180.0, -90.0]
      },
      "resolution": {
        "x": 0.1,
        "y": 0.1
      }
    }

result_descriptor = ge.RasterResultDescriptor(result_descriptor_dict)

meta_data = ge.GdalMetaDataStatic({
  "type": "GdalStatic",
  "time": None, 
  "params": gdal_params, 
  "resultDescriptor": result_descriptor
})

meta_data

{'type': 'GdalStatic',
 'time': None,
 'params': {'filePath': 'raster/landcover.tif',
  'rasterbandChannel': 1,
  'geoTransform': {'originCoordinate': [-180.0, 90.0],
   'xPixelSize': 0.1,
   'yPixelSize': -0.1},
  'width': 3600,
  'height': 1800,
  'fileNotFoundHandling': <FileNotFoundHandling.NODATA: 'NoData'>,
  'noDataValue': None,
  'propertiesMapping': None,
  'gdalOpenOptions': None,
  'gdalConfigOptions': None,
  'allowAlphabandAsMask': True},
 'resultDescriptor': Data type:         U8
 Spatial Reference: EPSG:4326
 Measurement:       Land Cover (0: Water Bodies, 1: Evergreen Needleleaf Forests, 2: Evergreen Broadleaf Forests, 3: Deciduous Needleleaf Forests, 4: Deciduous Broadleleaf Forests, 5: Mixed Forests, 6: Closed Shrublands, 7: Open Shrublands, 8: Woody Savannas, 9: Savannas, 10: Grasslands, 11: Permanent Wtlands, 12: Croplands, 13: Urban and Built-Up, 14: Cropland-Natural Vegetation Mosaics, 15: Snow and Ice, 16: Barren or Sparsely Vegetated)}

## Add raster dataset the the first volume

In [6]:
dataset_id = ge.add_public_raster_dataset(volumes[0].name, "landcover", meta_data)

dataset_id

{"dataPath": {"volume": "test_data"}, "definition": {"properties": {"name": "landcover", "description": "", "sourceOperator": "GdalSource"}, "metaData": {"type": "GdalStatic", "time": null, "params": {"filePath": "raster/landcover.tif", "rasterbandChannel": 1, "geoTransform": {"originCoordinate": [-180.0, 90.0], "xPixelSize": 0.1, "yPixelSize": -0.1}, "width": 3600, "height": 1800, "fileNotFoundHandling": "NoData", "noDataValue": null, "propertiesMapping": null, "gdalOpenOptions": null, "gdalConfigOptions": null, "allowAlphabandAsMask": true}, "resultDescriptor": {"type": "raster", "dataType": "U8", "measurement": {"type": "classification", "measurement": "Land Cover", "classes": {"0": "Water Bodies", "1": "Evergreen Needleleaf Forests", "2": "Evergreen Broadleaf Forests", "3": "Deciduous Needleleaf Forests", "4": "Deciduous Broadleleaf Forests", "5": "Mixed Forests", "6": "Closed Shrublands", "7": "Open Shrublands", "8": "Woody Savannas", "9": "Savannas", "10": "Grasslands", "11": "Pe

6610dfd1-048a-4dfa-85b4-9babe1d6de8c

## Verify that data is accessible for user

In [7]:
ge.reset()
ge.initialize("http://localhost:3030")

workflow = ge.register_workflow({
    "type": "Raster",
    "operator": {
        "type": "GdalSource",
        "params": {
            "data": {
                "type": "internal",
                "datasetId": str(dataset_id)
            },
        }
    }
})

workflow.get_result_descriptor()

Data type:         U8
Spatial Reference: EPSG:4326
Measurement:       Land Cover (0: Water Bodies, 1: Evergreen Needleleaf Forests, 10: Grasslands, 11: Permanent Wtlands, 12: Croplands, 13: Urban and Built-Up, 14: Cropland-Natural Vegetation Mosaics, 15: Snow and Ice, 16: Barren or Sparsely Vegetated, 2: Evergreen Broadleaf Forests, 3: Deciduous Needleleaf Forests, 4: Deciduous Broadleleaf Forests, 5: Mixed Forests, 6: Closed Shrublands, 7: Open Shrublands, 8: Woody Savannas, 9: Savannas)